In [ ]:
from datascience import *
import numpy as np
import pandas as pd

/usr/local/lib/python3.6/dist-packages/datascience/tables.py:17: MatplotlibDeprecationWarning: The 'warn' parameter of use() is deprecated since Matplotlib 3.1 and will be removed in 3.3.  If any parameter follows 'warn', they should be pass as keyword, not positionally.
  matplotlib.use('agg', warn=False)
/usr/local/lib/python3.6/dist-packages/datascience/util.py:10: MatplotlibDeprecationWarning: The 'warn' parameter of use() is deprecated since Matplotlib 3.1 and will be removed in 3.3.  If any parameter follows 'warn', they should be pass as keyword, not positionally.
  matplotlib.use('agg', warn=False)


In [ ]:
staff_demographics_table = Table.read_table('Staff Demographics.csv') # create a table of the staff demographics
staff_demographics_table

In [ ]:
# Drop the columns that we do not need
staff_updated = staff_demographics_table.drop(np.arange(3, 8)).drop(np.arange(4,7))
school_districts = ["Albany City Unified", "Alameda Unified", "Berkeley Unified", "Castro Valley Unified", "Dublin Unified",
                    "Emery Unified", "Fremont Unified", "Hayward Unified", "Livermore Unified", "Mountain House Elementary",
                    "New Haven Unified", "Newark Unified", "Oakland Unified", "Piedmont Unified", "Pleasanton Unified", "San Leandro Unified",
                    "San Lorenzo Unified", "Sunol Glen Unified", "San Francisco Unified"] # school districts in Alameda and San Francisco County (can also narrow this down to only Oakland and San Fran)
oakland_san_fran = ["Oakland Unified", "San Francisco Unified"] # the districts that we want to focus on right now
staff_updated = staff_updated.where("District", are.contained_in(oakland_san_fran))
staff_updated = staff_updated.relabel("African American not Hispanic", "African American")
staff_updated.show(15)

/usr/local/lib/python3.6/dist-packages/datascience/tables.py:699: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if i not in exclude and c not in exclude])


CDS,District,School,African American,Total
1.61259e+12,Oakland Unified,Achieve Academy,2,37
1.6126e+12,Oakland Unified,ACORN Woodland Elementary,1,20
1.6126e+12,Oakland Unified,Allendale Elementary,6,25
1.61259e+12,Oakland Unified,Alliance Academy,8,23
1.6126e+12,Oakland Unified,American Indian Public Charter,0,13
1.61259e+12,Oakland Unified,American Indian Public Charter II,5,43
1.61259e+12,Oakland Unified,American Indian Public High,1,18
1.61259e+12,Oakland Unified,ARISE High,1,17
1.6126e+12,Oakland Unified,ASCEND,1,26
1.61259e+12,Oakland Unified,Aspire Berkley Maynard Academy,5,23


In [ ]:
# calculate the percentage
staff_updated = staff_updated.with_column("Percentage Teachers",
                                          (staff_updated.column("African American")/staff_updated.column("Total"))*100)
final_staff = staff_updated.sort("Percentage Teachers").drop(0,3,4).where("School", are.containing("Elementary"))
final_staff

District,School,Percentage Teachers
San Francisco Unified,Alamo Elementary,0
San Francisco Unified,Alvarado Elementary,0
San Francisco Unified,Bryant Elementary,0
San Francisco Unified,Chavez (Cesar) Elementary,0
San Francisco Unified,Chin (John Yehall) Elementary,0
San Francisco Unified,Clarendon Alternative Elementary,0
San Francisco Unified,Cleveland Elementary,0
San Francisco Unified,Cobb (William L.) Elementary,0
San Francisco Unified,Feinstein (Dianne) Elementary,0
San Francisco Unified,Flynn (Leonard R.) Elementary,0


In [ ]:
enrollment = Table.read_table('Student Enrollment final.csv') # create a table of the enrollment data
enrollment = enrollment.drop(np.arange(0,4)).drop(np.arange(2,12)).drop(np.arange(4,11))
enrollment_updated = enrollment.where("District", are.contained_in(oakland_san_fran)).relabel("PctAfAm", "Percentage Students").drop(2)
enrollment_updated

/usr/local/lib/python3.6/dist-packages/datascience/tables.py:699: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if i not in exclude and c not in exclude])


District,School,Percentage Students
Oakland Unified,Achieve Academy,3.38
Oakland Unified,ACORN Woodland Elementary,4.63
Oakland Unified,Allendale Elementary,27.75
Oakland Unified,American Indian Public Charter,24.46
Oakland Unified,American Indian Public Charter II,34.92
Oakland Unified,American Indian Public High,24.11
Oakland Unified,ARISE High,1.9
Oakland Unified,ASCEND,1.62
Oakland Unified,Aspire Berkley Maynard Academy,57.89
Oakland Unified,Aspire College Academy,29.01


In [ ]:
# combine the two tables together

staff_and_students = final_staff.join("School", enrollment_updated, "School").drop(3)
# calculate the difference between the two percentages
final_staff_and_students = staff_and_students.with_column("Difference",
                          staff_and_students.column("Percentage Students") - staff_and_students.column("Percentage Teachers")) # difference in the two percentages
final_staff_and_students = final_staff_and_students.sort("Difference", descending = True)
final_staff_and_students

School,District,Percentage Teachers,Percentage Students,Difference
Cobb (William L.) Elementary,San Francisco Unified,0,30.88,30.88
Harte (Bret) Elementary,San Francisco Unified,0,23.96,23.96
Grass Valley Elementary,Oakland Unified,44.4444,63.78,19.3356
Garfield Elementary,San Francisco Unified,0,19.07,19.07
El Dorado Elementary,San Francisco Unified,9.09091,27.39,18.2991
Emerson Elementary,Oakland Unified,26.9231,44.27,17.3469
Cleveland Elementary,San Francisco Unified,0,17.13,17.13
Burckhalter Elementary,Oakland Unified,50,66.22,16.22
King (Thomas Starr) Elementary,San Francisco Unified,0,14.66,14.66
Kaiser Elementary,Oakland Unified,7.14286,21.13,13.9871


In [ ]:
school_addresses = Table.read_table('School Addresses.csv') # create a table of the school addresses
school_addresses.show(2)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


CDSCode,NCESDist,NCESSchool,StatusType,County,District,School,Street,StreetAbr,City,Zip,State,MailStreet,MailStrAbr,MailCity,MailZip,MailState,Phone,Ext,FaxNumber,Email,WebSite,OpenDate,ClosedDate,Charter,CharterNum,FundingType,DOC,DOCType,SOC,SOCType,EdOpsCode,EdOpsName,EILCode,EILName,GSoffered,GSserved,Virtual,Magnet,YearRoundYN,FederalDFCDistrictID,Latitude,Longitude,AdmFName,AdmLName,AdmEmail,LastUpDate
1100170000000.0,691051,No Data,Active,Alameda,Alameda County Office of Education,No Data,313 West Winton Avenue,313 West Winton Ave.,Hayward,94544-1136,CA,313 West Winton Avenue,313 West Winton Ave.,Hayward,94544-1136,CA,(510) 887-0152,No Data,No Data,No Data,http://www.acoe.org,No Data,No Data,No Data,No Data,No Data,0,County Office of Education (COE),No Data,No Data,No Data,No Data,No Data,No Data,No Data,No Data,No Data,No Data,No Data,No Data,37.658212,-122.09713,L Karen,Monroe,lkmonroe@acoe.org,8/15/20
1100170000000.0,691051,10546,Closed,Alameda,Alameda County Office of Education,FAME Public Charter,"39899 Balentine Drive, Suite 335","39899 Balentine Dr., Ste. 335",Newark,94560-5359,CA,"39899 Balentine Drive, Suite 335","39899 Balentine Dr., Ste. 335",Newark,94560-5359,CA,No Data,No Data,No Data,malik@famecharter.org,No Data,8/29/05,7/31/15,Y,728,Directly funded,0,County Office of Education (COE),65,K-12 Schools (Public),TRAD,Traditional,ELEMHIGH,Elementary-High Combination,K-12,K-12,P,N,N,No Data,37.521436,-121.99391,No Data,No Data,No Data,9/1/15


In [ ]:
school_addresses_edited = school_addresses.select(5,6,7,8,9,10,11,41,42)
school_addresses_edited = school_addresses_edited.where("District", are.contained_in(oakland_san_fran))
school_addresses_edited

District,School,Street,StreetAbr,City,Zip,State,Latitude,Longitude
Oakland Unified,No Data,"1000 Broadway, Suite 300","1000 Broadway, Ste. 300",Oakland,94607-4099,CA,37.801387,-122.27194
Oakland Unified,Oakland Unity High,6038 Brann Street,6038 Brann St.,Oakland,94605-1544,CA,37.773706,-122.18548
Oakland Unified,East Oakland Leadership Academy,2614 Seminary Avenue,2614 Seminary Ave.,Oakland,94605-1570,CA,37.77184,-122.18986
Oakland Unified,ACORN Woodland Elementary,1025 - 81st Avenue,1025 - 81st Ave.,Oakland,94621,CA,No Data,No Data
Oakland Unified,MetWest High,314 East Tenth Street,314 East Tenth St.,Oakland,94606-2302,CA,37.795107,-122.25823
Oakland Unified,The San Antonio Multicultural,"1025 Second Avenue, Room 301","1025 Second Ave., Room 301",Oakland,94606,CA,No Data,No Data
Oakland Unified,Think College Now,2825 International Boulevard,2825 International Blvd.,Oakland,94601-1543,CA,37.78008,-122.2305
Oakland Unified,Mandela High,4610 Foothill Boulevard,4610 Foothill Blvd.,Oakland,94601,CA,37.773786,-122.20992
Oakland Unified,Media College Preparatory,4610 Foothill Boulevard,4610 Foothill Blvd.,Oakland,94601-4618,CA,37.773786,-122.20992
Oakland Unified,Robeson School of Visual and Performing Arts,4610 Foothill Boulevard,4610 Foothill Blvd.,Oakland,94601,CA,37.773786,-122.20992


In [ ]:
count = final_staff_and_students.join("School", school_addresses_edited, "School").group("School")
count.where("count", are.not_equal_to(1))
repeats_list = ["ACORN Woodland Elementary", "Jefferson Elementary", "Longfellow Elementary", "Marshall Elementary", "Sherman Elementary",
                "Cleveland Elementary", "Garfield Elementary", "Hillcrest Elementary"]

In [ ]:
def first(values):
  return values.item(0)
final_staff_and_students = final_staff_and_students.sort("School").group(["District", "School"], first)
school_addresses_edited = school_addresses_edited.sort("School").group(["District", "School"], first)
joint_tables = final_staff_and_students.join("School", school_addresses_edited, "School")
bad_rows = make_array()
for row in np.arange(joint_tables.num_rows):
  if joint_tables.column(1).item(row) != joint_tables.column(5).item(row):
    bad_rows = np.append(bad_rows, row)
joint_tables = joint_tables.remove(16).remove(16).remove(31).remove(31).remove(40).remove(40).remove(45).remove(
    58).remove(61).remove(87) #,17,33,34,44,45,51,65,69,96)
joint_tables.show()

School,District,Percentage Teachers first,Percentage Students first,Difference first,District_2,Street first,StreetAbr first,City first,Zip first,State first,Latitude first,Longitude first
ACORN Woodland Elementary,Oakland Unified,5,4.63,-0.37,Oakland Unified,1025 - 81st Avenue,1025 - 81st Ave.,Oakland,94621,CA,No Data,No Data
Alamo Elementary,San Francisco Unified,0,0.57,0.57,San Francisco Unified,250 23rd Avenue,250 23rd Ave.,San Francisco,94121-2009,CA,37.783124,-122.48219
Allendale Elementary,Oakland Unified,24,27.75,3.75,Oakland Unified,3670 Penniman Avenue,3670 Penniman Ave.,Oakland,94619-1116,CA,37.788642,-122.20516
Alvarado Elementary,San Francisco Unified,0,1.58,1.58,San Francisco Unified,625 Douglas Street,625 Douglas St.,San Francisco,94114-3140,CA,37.753675,-122.43873
Argonne Elementary,San Francisco Unified,5.26316,1.55,-3.71316,San Francisco Unified,680 18th Avenue,680 18th Ave.,San Francisco,94121-3818,CA,37.77515,-122.47679
Bella Vista Elementary,Oakland Unified,13.7931,19.66,5.8669,Oakland Unified,1025 East 28th Street,1025 East 28th St.,Oakland,94610-4018,CA,37.800355,-122.23708
Brookfield Elementary,Oakland Unified,26.6667,24.54,-2.12667,Oakland Unified,401 Jones Avenue,401 Jones Ave.,Oakland,94603-1123,CA,37.734711,-122.189
Bryant Elementary,San Francisco Unified,0,6.41,6.41,San Francisco Unified,2641 25th Street,2641 25th St.,San Francisco,94110-3514,CA,37.751628,-122.4049
Burckhalter Elementary,Oakland Unified,50,66.22,16.22,Oakland Unified,3994 Burckhalter Avenue,3994 Burckhalter Ave.,Oakland,94605-2616,CA,37.775578,-122.16679
Carl B. Munck Elementary,Oakland Unified,47.0588,51.29,4.23118,Oakland Unified,11900 Campus Drive,11900 Campus Dr.,Oakland,94619-3104,CA,37.794753,-122.17382


In [ ]:
joint_tables = joint_tables.drop(5).group(["District", "School"], first)
joint_tables

District,School,Percentage Teachers first first,Percentage Students first first,Difference first first,Street first first,StreetAbr first first,City first first,Zip first first,State first first,Latitude first first,Longitude first first
Oakland Unified,ACORN Woodland Elementary,5,4.63,-0.37,1025 - 81st Avenue,1025 - 81st Ave.,Oakland,94621,CA,No Data,No Data
Oakland Unified,Allendale Elementary,24,27.75,3.75,3670 Penniman Avenue,3670 Penniman Ave.,Oakland,94619-1116,CA,37.788642,-122.20516
Oakland Unified,Bella Vista Elementary,13.7931,19.66,5.8669,1025 East 28th Street,1025 East 28th St.,Oakland,94610-4018,CA,37.800355,-122.23708
Oakland Unified,Brookfield Elementary,26.6667,24.54,-2.12667,401 Jones Avenue,401 Jones Ave.,Oakland,94603-1123,CA,37.734711,-122.189
Oakland Unified,Burckhalter Elementary,50,66.22,16.22,3994 Burckhalter Avenue,3994 Burckhalter Ave.,Oakland,94605-2616,CA,37.775578,-122.16679
Oakland Unified,Carl B. Munck Elementary,47.0588,51.29,4.23118,11900 Campus Drive,11900 Campus Dr.,Oakland,94619-3104,CA,37.794753,-122.17382
Oakland Unified,Chabot Elementary,8.57143,7.34,-1.23143,6686 Chabot Road,6686 Chabot Rd.,Oakland,94618-1641,CA,37.850664,-122.24293
Oakland Unified,Cleveland Elementary,33.3333,17.13,-16.2033,745 Cleveland Street,745 Cleveland St.,Oakland,94606-1513,CA,37.804369,-122.24405
Oakland Unified,Community United Elementary,26.087,11.53,-14.557,6701 International Boulevard,6701 International Blvd.,Oakland,94621-3501,CA,37.761424,-122.19188
Oakland Unified,Crocker Highlands Elementary,14.2857,7.42,-6.86571,525 Midcrest Road,525 Midcrest Rd.,Oakland,94610-1813,CA,37.811364,-122.22738


In [ ]:
joint_tables = joint_tables.sort("Difference first first", descending = True)
relabel_old = ["Percentage Teachers first first", "Percentage Students first first", "Difference first first",
               "Street first first", "StreetAbr first first", "City first first", "Zip first first", "State first first",
               "Latitude first first", "Longitude first first"]
relabel_new = ["Percentage Teachers", "Percentage Students", "Difference",
               "Street", "StreetAbr", "City", "Zip", "State","Latitude", "Longitude"]
joint_tables = joint_tables.relabel(relabel_old[0], relabel_new[0]).relabel(relabel_old[1], relabel_new[1]).relabel(
    relabel_old[2], relabel_new[2]).relabel(relabel_old[3], relabel_new[3]).relabel(relabel_old[4], relabel_new[4]).relabel(
    relabel_old[5], relabel_new[5]).relabel(relabel_old[6], relabel_new[6]).relabel(relabel_old[7], relabel_new[7]).relabel(
    relabel_old[8], relabel_new[8]).relabel(relabel_old[9], relabel_new[9])
joint_tables

District,School,Percentage Teachers,Percentage Students,Difference,Street,StreetAbr,City,Zip,State,Latitude,Longitude
San Francisco Unified,Cobb (William L.) Elementary,0,30.88,30.88,2725 California Street,2725 California St.,San Francisco,94115-2513,CA,37.788174,-122.43918
San Francisco Unified,Harte (Bret) Elementary,0,23.96,23.96,1035 Gilman Avenue,1035 Gilman Ave.,San Francisco,94124-3710,CA,37.71861,-122.38889
Oakland Unified,Grass Valley Elementary,44.4444,63.78,19.3356,4720 Dunkirk Avenue,4720 Dunkirk Ave.,Oakland,94605-5607,CA,37.75301,-122.12585
San Francisco Unified,Garfield Elementary,0,19.07,19.07,420 Filbert Street,420 Filbert St.,San Francisco,94133-3002,CA,37.8018,-122.40673
San Francisco Unified,El Dorado Elementary,9.09091,27.39,18.2991,70 Delta Street,70 Delta St.,San Francisco,94134-2145,CA,37.719265,-122.40673
Oakland Unified,Emerson Elementary,26.9231,44.27,17.3469,4803 Lawton Avenue,4803 Lawton Ave.,Oakland,94609-2208,CA,37.834628,-122.2573
San Francisco Unified,Cleveland Elementary,0,17.13,17.13,455 Athens Street,455 Athens St.,San Francisco,94112-2801,CA,37.720733,-122.42904
Oakland Unified,Burckhalter Elementary,50,66.22,16.22,3994 Burckhalter Avenue,3994 Burckhalter Ave.,Oakland,94605-2616,CA,37.775578,-122.16679
San Francisco Unified,King (Thomas Starr) Elementary,0,14.66,14.66,1215 Carolina Street,1215 Carolina St.,San Francisco,94107-3322,CA,37.752765,-122.39935
Oakland Unified,Kaiser Elementary,7.14286,21.13,13.9871,25 South Hill Court,25 South Hill Ct.,Oakland,94618-2306,CA,37.853077,-122.23181


In [ ]:
#joint_tables.to_df().to_csv('joint_tables.csv', index = False)

In [ ]:
joint_tables = joint_tables.drop(6)

In [ ]:
joint_tables

District,School,Percentage Teachers,Percentage Students,Difference,Street,City,Zip,State,Latitude,Longitude
San Francisco Unified,Cobb (William L.) Elementary,0,30.88,30.88,2725 California Street,San Francisco,94115-2513,CA,37.788174,-122.43918
San Francisco Unified,Harte (Bret) Elementary,0,23.96,23.96,1035 Gilman Avenue,San Francisco,94124-3710,CA,37.71861,-122.38889
Oakland Unified,Grass Valley Elementary,44.4444,63.78,19.3356,4720 Dunkirk Avenue,Oakland,94605-5607,CA,37.75301,-122.12585
San Francisco Unified,Garfield Elementary,0,19.07,19.07,420 Filbert Street,San Francisco,94133-3002,CA,37.8018,-122.40673
San Francisco Unified,El Dorado Elementary,9.09091,27.39,18.2991,70 Delta Street,San Francisco,94134-2145,CA,37.719265,-122.40673
Oakland Unified,Emerson Elementary,26.9231,44.27,17.3469,4803 Lawton Avenue,Oakland,94609-2208,CA,37.834628,-122.2573
San Francisco Unified,Cleveland Elementary,0,17.13,17.13,455 Athens Street,San Francisco,94112-2801,CA,37.720733,-122.42904
Oakland Unified,Burckhalter Elementary,50,66.22,16.22,3994 Burckhalter Avenue,Oakland,94605-2616,CA,37.775578,-122.16679
San Francisco Unified,King (Thomas Starr) Elementary,0,14.66,14.66,1215 Carolina Street,San Francisco,94107-3322,CA,37.752765,-122.39935
Oakland Unified,Kaiser Elementary,7.14286,21.13,13.9871,25 South Hill Court,Oakland,94618-2306,CA,37.853077,-122.23181


In [ ]:
for_lunch_data_table = Table.read_table('free or reduced lunch.csv') # create a table of the f.o.r lunch data
for_lunch_data_table = for_lunch_data_table.drop(np.arange(0,5)).drop(np.arange(2,14)).drop(np.arange(3,11))
for_lunch_data_table = for_lunch_data_table.where("District Name", are.contained_in(oakland_san_fran))
for_lunch_data_table = for_lunch_data_table.sort('Percent (%) \nEligible Free \n(K-12)', descending=True).where("School Name", are.containing("Elementary"))

for_lunch_data_table = for_lunch_data_table.relabel(2, "Percentage").group("School Name", first)
for_lunch_data_table

/usr/local/lib/python3.6/dist-packages/datascience/tables.py:699: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if i not in exclude and c not in exclude])


School Name,District Name first,Percentage first
ACORN Woodland Elementary,Oakland Unified,84.00%
Alamo Elementary,San Francisco Unified,30.70%
Allendale Elementary,Oakland Unified,77.30%
Alvarado Elementary,San Francisco Unified,21.90%
Argonne Elementary,San Francisco Unified,21.00%
Bella Vista Elementary,Oakland Unified,68.20%
Brookfield Elementary,Oakland Unified,79.60%
Bryant Elementary,San Francisco Unified,75.60%
Burckhalter Elementary,Oakland Unified,82.00%
Carl B. Munck Elementary,Oakland Unified,66.40%


In [ ]:
final_table = joint_tables.join("School", for_lunch_data_table, "School Name").drop(11)
for string in np.arange(len(final_table.column("Percentage first"))):
  final_table.column("Percentage first")[string] = final_table.column("Percentage first")[string].replace("%", "")
final_table = final_table.sort("Difference", descending=True).relabel("Percentage Teachers", "Teachers (%)").relabel(
                  "Percentage Students", "Students (%)").relabel( "Difference", "Difference: Student - Teacher (%)").relabel("Percentage first",
                  "Free/Reduced Lunch(%)").move_to_start("Free/Reduced Lunch(%)").move_to_start("School").move_to_start("District")
final_table

District,School,Free/Reduced Lunch(%),Teachers (%),Students (%),Difference: Student - Teacher (%),Street,City,Zip,State,Latitude,Longitude
San Francisco Unified,Cobb (William L.) Elementary,56.60,0,30.88,30.88,2725 California Street,San Francisco,94115-2513,CA,37.788174,-122.43918
San Francisco Unified,Harte (Bret) Elementary,74.20,0,23.96,23.96,1035 Gilman Avenue,San Francisco,94124-3710,CA,37.71861,-122.38889
Oakland Unified,Grass Valley Elementary,73.60,44.4444,63.78,19.3356,4720 Dunkirk Avenue,Oakland,94605-5607,CA,37.75301,-122.12585
San Francisco Unified,Garfield Elementary,84.30,0,19.07,19.07,420 Filbert Street,San Francisco,94133-3002,CA,37.8018,-122.40673
San Francisco Unified,El Dorado Elementary,70.10,9.09091,27.39,18.2991,70 Delta Street,San Francisco,94134-2145,CA,37.719265,-122.40673
Oakland Unified,Emerson Elementary,65.30,26.9231,44.27,17.3469,4803 Lawton Avenue,Oakland,94609-2208,CA,37.834628,-122.2573
San Francisco Unified,Cleveland Elementary,70.90,0,17.13,17.13,455 Athens Street,San Francisco,94112-2801,CA,37.720733,-122.42904
Oakland Unified,Burckhalter Elementary,82.00,50,66.22,16.22,3994 Burckhalter Avenue,Oakland,94605-2616,CA,37.775578,-122.16679
San Francisco Unified,King (Thomas Starr) Elementary,40.20,0,14.66,14.66,1215 Carolina Street,San Francisco,94107-3322,CA,37.752765,-122.39935
Oakland Unified,Kaiser Elementary,30.90,7.14286,21.13,13.9871,25 South Hill Court,Oakland,94618-2306,CA,37.853077,-122.23181


In [ ]:
final_table.to_df().to_csv('final_table.csv', index = False)